# Pipeline

1. Preprocessing
2. **Data Extraction**
3. Data Exploration
4. Model

This file performs data extraction. It generates several features and returns a .pkl file containing a dataframe with said features.

# Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict

# Data extraction into new Dataframe

In [2]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
results = pd.read_pickle('preprocessed_results.pkl')
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,1
1,1,12,1,0,1
2,2,16,0,0,0
3,3,14,6,0,1
4,4,11,2,2,0
...,...,...,...,...,...
13007,97,105,2,2,0
13008,111,106,2,0,1
13009,120,109,0,2,2
13010,122,107,2,2,0


In [3]:
df = pd.read_pickle('feature_frame.pkl')

In [15]:
# Only run this if the above cell hasn't been run!!
df['Date'] = results['Date']
df['league'] = results['Div']

In [6]:
results.sample(3)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1056,E1,2013-03-30,10,23,4,1,1,1,0,H,...,14,2,9,14,11,3,0,1,0,0
1377,E1,2014-02-02,10,25,2,0,1,0,0,D,...,6,2,9,14,6,5,1,2,0,0
1371,E1,2014-02-01,12,0,2,1,1,0,1,A,...,6,1,4,18,13,1,1,0,0,0


In [5]:
# "Constants"
suffixes = {'HomeID':'_home', 'AwayID':'_away'}    # suffix dict for the feature names

id_names = ['HomeID', 'AwayID']                    # list of ID columns for iteration purposes

# suffix that's appended to all league-wide features
league_suffix = '_league'

In [5]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

# Implementing non-relational Features from the Hubáček et al. paper

__For the moment, the features are closely aligned to those implemented by Hubáček et al. That means that most features also get implemented twice, once for the home team and once for that away team. The naming convention for this approach is FEATURE_NAME_home/away.__

Example: <br>
A_WIN_PCT_home would be the historical away winning percentage of the team that plays the current game at home. <br>
H_WIN_PCT_home would be the historical home winning percentage of that same team.

That means the word after the last underscore indicates if the value of this feature represents the home team or the away team of the current(!) game.

The current approach is to prefer readability over performance which means that it can take long to add the features to the dataframe.

In [43]:
# TODO: evtl. die feature-erstellungen in Funktionen packen um sie an und aus togglen zu können
# Alle nicht-kursiven Features aus dem Hubacek Paper müssen doppelt implementiert werden, siehe oben.

## Historical strength

In [18]:
# Returns all games that were played by a specific team between date minus two years and date.
def get_historical_games(team_id, date):
    games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games_by_team

In [19]:
get_historical_games(8, datetime.date(2016, 5, 20)).head(6)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1527,E1,2014-08-17,8,24,2,1,1,1,0,H,...,5,6,8,11,2,6,1,2,0,0
1539,E1,2014-08-25,18,8,3,1,1,1,0,H,...,4,3,13,7,6,7,1,1,0,0
1549,E1,2014-08-31,5,8,0,3,2,0,1,A,...,1,8,14,11,7,3,0,4,0,0
1553,E1,2014-09-13,8,0,0,1,2,0,1,A,...,1,1,9,10,7,6,2,1,0,0
1565,E1,2014-09-20,15,8,3,1,1,2,1,H,...,7,5,16,11,7,5,4,1,0,0
1574,E1,2014-09-27,8,12,1,1,0,0,0,D,...,8,5,8,9,8,2,2,1,0,0


### Winning and drawing percentages

In [20]:
# Add percentages to dataframe
df['H_WIN_PCT_home'] = 0
df['H_DRAW_PCT_home'] = 0
df['A_WIN_PCT_home'] = 0
df['A_DRAW_PCT_home'] = 0
df['H_WIN_PCT_away'] = 0
df['H_DRAW_PCT_away'] = 0
df['A_WIN_PCT_away'] = 0
df['A_DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)

        # total counts of home and away games in the last two years
        home_game_count = len(hist_games[hist_games['HomeID'] == team_id])
        away_game_count = len(hist_games[hist_games['AwayID'] == team_id])

        home_games_won = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 1)])
        away_games_won = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 2)])

        home_games_drawn = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 0)])
        away_games_drawn = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 0)])

        if home_game_count > 0:         # TODO should the threshold be higher to only use significant enough data?
            df.loc[row, 'H_WIN_PCT' + suffix] = home_games_won / home_game_count
            df.loc[row, 'H_DRAW_PCT' + suffix] = home_games_drawn / home_game_count
        else:
            df.loc[row, 'H_WIN_PCT' + suffix] = 0          # TODO maybe choose some standard value like 0.25 instead of 0?
            df.loc[row, 'H_DRAW_PCT' + suffix] = 0         # Overall Median/Mean could also work.

        if away_game_count > 0:
            df.loc[row, 'A_WIN_PCT' + suffix] = away_games_won / away_game_count
            df.loc[row, 'A_DRAW_PCT' + suffix] = away_games_drawn / away_game_count
        else:
            df.loc[row, 'A_WIN_PCT' + suffix] = 0          
            df.loc[row, 'A_DRAW_PCT' + suffix] = 0 

### Goal averages and standard deviations

In [23]:
# Add goal averages and deviations to dataframe
df['H_GS_AVG_home'] = 0
df['H_GC_AVG_home'] = 0
df['A_GS_AVG_home'] = 0
df['A_GC_AVG_home'] = 0
df['H_GS_AVG_away'] = 0
df['H_GC_AVG_away'] = 0
df['A_GS_AVG_away'] = 0
df['A_GC_AVG_away'] = 0

df['H_GS_STD_home'] = 0
df['H_GC_STD_home'] = 0
df['A_GS_STD_home'] = 0
df['A_GC_STD_home'] = 0
df['H_GS_STD_away'] = 0
df['H_GC_STD_away'] = 0
df['A_GS_STD_away'] = 0
df['A_GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)
        
        # calculate averages
        df.loc[row, 'H_GS_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].mean()
        df.loc[row, 'H_GC_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GS_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GC_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].mean()
        
        # calculate standard deviations
        df.loc[row, 'H_GS_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].std()
        df.loc[row, 'H_GC_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GS_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GC_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].std()
        
# During this process, many NaNs occur in the beginning due to insufficient data.
# We replace them with 0.
df.fillna(0, inplace=True)

In [31]:
df.sample(5)

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,A_GS_AVG_away,A_GC_AVG_away,H_GS_STD_home,H_GC_STD_home,A_GS_STD_home,A_GC_STD_home,H_GS_STD_away,H_GC_STD_away,A_GS_STD_away,A_GC_STD_away
6101,51,49,1,3,2,2019-03-17,0.454545,0.272727,0.416667,0.222222,...,1.500000,1.558824,1.136515,1.314978,1.769898,1.133543,1.231246,1.379319,1.212311,1.049998
7430,78,81,3,0,1,2013-11-10,0.763158,0.184211,0.625000,0.275000,...,1.552632,1.157895,1.109893,0.724004,1.347124,0.921259,1.413987,1.025008,1.288145,1.000711
6634,76,81,2,1,1,2011-10-29,0.565217,0.260870,0.043478,0.391304,...,1.391304,1.130435,0.934622,1.083473,0.886883,1.029217,1.222322,1.042572,1.117592,1.057628
7268,68,74,1,0,1,2013-04-28,0.405405,0.270270,0.078947,0.368421,...,1.324324,1.405405,1.437904,1.168917,1.010964,1.148937,1.411445,1.172225,1.055516,1.300843
12964,120,127,1,0,1,2019-04-23,0.187500,0.312500,0.235294,0.352941,...,1.228571,1.628571,0.885061,1.264911,0.951006,1.169464,1.222475,1.171080,0.972738,1.628527


## Current form (data from the last five games)

In [11]:
# df_mode indicates whether to return the df dataframe or the results dataframe
def get_last_n_games(team_id, date, n=5, df_mode=False):
    if df_mode == False:
        games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
    else:
        games_by_team = df[((df['HomeID'] == team_id) | (df['AwayID'] == team_id))
                           & (df['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
        
    return games_by_team

In [33]:
get_last_n_games(19, datetime.date(2011, 6, 1))

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
374,E1,2011-05-22,9,19,4,2,1,1,1,D,...,11,7,8,5,8,8,2,1,0,0
361,E1,2011-05-14,19,2,4,3,1,3,2,H,...,8,9,16,12,5,6,2,4,0,0
353,E1,2011-05-07,5,19,1,1,0,0,0,D,...,11,7,5,13,7,6,3,2,0,0
340,E1,2011-04-30,19,13,0,0,0,0,0,D,...,6,6,10,11,4,8,0,0,0,0
328,E1,2011-04-23,19,17,1,1,0,1,1,D,...,7,6,17,8,4,4,1,1,0,0


### Winning and drawing percentages

In [34]:
df['WIN_PCT_home'] = 0
df['WIN_PCT_away'] = 0
df['DRAW_PCT_home'] = 0
df['DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        recent = get_last_n_games(team_id, game_date)
        #print(len(recent))
        # game_count could be less than five if not enough data is available; that's the only reason for this line
        game_count = len(recent)
        
        games_won = len(recent[(recent['HomeID'] == team_id) & (recent['FTR'] == 1)
                    | (recent['AwayID'] == team_id) & (recent['FTR'] == 2)])
        
        games_drawn = len(recent[recent['FTR'] == 0])
        
        suffix = suffixes[id_name]
        
        if game_count > 0:
            df.loc[row, 'WIN_PCT' + suffix] = games_won / game_count
            df.loc[row, 'DRAW_PCT' + suffix] = games_drawn / game_count
        else:
            df.loc[row, 'WIN_PCT' + suffix] = 0
            df.loc[row, 'DRAW_PCT' + suffix] = 0
            

### Goal averages and standard deviations

In [35]:
df['GS_AVG_home'] = 0
df['GC_AVG_home'] = 0
df['GS_AVG_away'] = 0
df['GC_AVG_away'] = 0

df['GS_STD_home'] = 0
df['GC_STD_home'] = 0
df['GS_STD_away'] = 0
df['GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        if game_count > 0:    # TODO gucken ob man vielleicht höheren threshold nehmen will und dann einfach mean assigned
            df.loc[row, 'GS_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTHG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTAG'].sum())) / game_count

            df.loc[row, 'GC_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTAG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTHG'].sum())) / game_count
            
            df.loc[row, 'GS_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTHG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTAG'].tolist()))
        
            df.loc[row, 'GC_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTAG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTHG'].tolist()))
            
        else:
            df.loc[row, 'GS_AVG' + suffix] = 0 
            df.loc[row, 'GC_AVG' + suffix] = 0
            df.loc[row, 'GS_STD' + suffix] = 0 
            df.loc[row, 'GC_STD' + suffix] = 0

### Rest days

In [53]:
df['REST_home'] = -1
df['REST_away'] = -1

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]

        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        # overall not the nicest solution, but works
        if game_count > 0:
            last_game = get_last_n_games(team_id, game_date).head(1)     
            try:
                rest_days = (game_date - last_game.iloc[0, 1]).days      # 0 is row, 1 is column (date)
            except:
                rest_days = -1
            
            if rest_days < 0 or rest_days > 21:             # manipulate outliers
                rest_days = 7
            
            df.loc[row, 'REST' + suffix] = rest_days
        else:
            df.loc[row, 'REST' + suffix] = 7    # if now data is available, just one week of rest

## League averages

In [54]:
def get_league_games(league, date):
    games = results[(results['Div'] == league) & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games

In [55]:
df['H_GS_AVG' + league_suffix] = 0
df['A_GS_AVG' + league_suffix] = 0
df['H_GS_STD' + league_suffix] = 0
df['A_GS_STD' + league_suffix] = 0
df['H_WIN_PCT' + league_suffix] = 0
df['DRAW_PCT' + league_suffix] = 0

for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']

    league_games = get_league_games(league_name, game_date)
    game_count = len(league_games)

    # goals scored means
    df.loc[row, 'H_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].mean()
    df.loc[row, 'A_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].mean()
    
    # goals scored standard deviations
    df.loc[row, 'H_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].std()
    df.loc[row, 'A_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].std()
   
    home_games_won = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 1)])
    games_drawn = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 0)])
   
    if game_count > 0:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = home_games_won / game_count
        df.loc[row, 'DRAW_PCT' + league_suffix] = games_drawn / game_count
    else:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = 0
        df.loc[row, 'DRAW_PCT' + league_suffix] = 0

### Other league features

In [60]:
df['TEAM_CNT' + league_suffix] = 0
df['GD_STD' + league_suffix] = 0
df['RND_CNT' + league_suffix] = 0
for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']
    
    # all leagues consist of 20 teams, except the Bundesliga (D1)
    if league_name == 'D1':
        df.loc[row, 'TEAM_CNT' + league_suffix] = 18
        df.loc[row, 'RND_CNT' + league_suffix] = 34
    else:
        df.loc[row, 'TEAM_CNT' + league_suffix] = 20
        df.loc[row, 'RND_CNT' + league_suffix] = 38
        
    games = get_league_games(league_name, game_date)
    
    # goal difference standard deviation
    df.loc[row, 'GD_STD' + league_suffix] = abs(games['FTHG'] - games['FTAG']).std() # muss hier wirklich abs() hin?

## Pi-ratings

### Functions

In [61]:
# Constant and error function (called psi) from the pi-ratings paper by Constantinou and Fenton
c = 3
def psi(error):
    return c * np.log10(1 + error)

def psi_h(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) < 0:
        return psi(error)
    else:
        return -1 * psi(error)

def psi_a(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) > 0:
        return psi(error)
    else:
        return -1 * psi(error)
    

# Constant and goal difference prediction function
b = 10
def predict_gd(rating):
    if rating >= 0:
        return np.power(b, np.abs(rating) / c) - 1
    else:
        return -(np.power(b, np.abs(rating) / c) - 1)

'''
Returns a dict with keys H_RTG_home, H_RTG_away, A_RTG_away, A_RTG_away
Used for one game!

cur_ratings: input in the same form as the output (dict with 4 elems); current ratings
'''
def update_pi_ratings(cur_ratings, actual_gd):
    H_RTG_home, H_RTG_away, A_RTG_home, A_RTG_away = cur_ratings['H_RTG_home'], cur_ratings['H_RTG_away'], cur_ratings['A_RTG_home'], cur_ratings['A_RTG_away']
    
    exp_gd = predict_gd(H_RTG_home) - predict_gd(A_RTG_away)    # Calculate expected goal difference
    
    error = np.abs(actual_gd - exp_gd)
    
    # The two learning rates from the pi-ratings paper (may have to be adjusted)
    lr_lambda = 0.1
    lr_gamma = 0.3
    
    # Update the pi-ratings
    new_H_RTG_home = H_RTG_home + psi_h(error, exp_gd, actual_gd) * lr_lambda 
    new_H_RTG_away = H_RTG_away + (new_H_RTG_home - H_RTG_home) * lr_gamma
    
    new_A_RTG_away = A_RTG_away + psi_a(error, exp_gd, actual_gd) * lr_lambda
    new_A_RTG_home = A_RTG_home + (new_A_RTG_away - A_RTG_away) * lr_gamma
    
    new_ratings = {'H_RTG_home': new_H_RTG_home, 'H_RTG_away': new_H_RTG_away, 'A_RTG_away': new_A_RTG_away, 'A_RTG_home': new_A_RTG_home}
    
    return new_ratings

In [62]:
update_pi_ratings({'H_RTG_home': 1.6, 'H_RTG_away': 0.4, 'A_RTG_home': 0.3, 'A_RTG_away': -1.2}, 3)

{'H_RTG_home': 1.5145736713398419,
 'H_RTG_away': 0.37437210140195254,
 'A_RTG_away': -1.1145736713398415,
 'A_RTG_home': 0.3256278985980475}

In [63]:
predict_gd(-1.5)

-2.1622776601683795

### Implementation into dataframe

In [64]:
# Returns the ID of the next game of a given team.
def get_next_game_id(team_id, date):
    games = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] > date)].sort_values(by='Date')
    
    if len(games) > 0:
        return games.index[0]    # [0] refers to the first element of the array returned by .index here
    else:
        print("LAST GAME:", date, team_id)
        return None

In [66]:
get_next_game_id(31, datetime.date(2017, 4, 2))

2576

In [ ]:
df['H_RTG_home'] = 0
df['A_RTG_home'] = 0
df['H_RTG_away'] = 0
df['A_RTG_away'] = 0

for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']    
    
    # Get current pi ratings from dataframe
    cur_ratings = {
        'H_RTG_home': df.loc[row, 'H_RTG_home'],
        'H_RTG_away': df.loc[row, 'H_RTG_away'],
        'A_RTG_home': df.loc[row, 'A_RTG_home'],
        'A_RTG_away': df.loc[row, 'A_RTG_away']
    }
    
    actual_gd = game['FTHG'] - game['FTAG']
    
    new_ratings = update_pi_ratings(cur_ratings, actual_gd)
    
    # Update all pi ratings for the next game (if it wasn't a teams last game)

    next_game_id_H = get_next_game_id(game['HomeID'], game_date)
    next_game_id_A = get_next_game_id(game['AwayID'], game_date)
    
    if next_game_id_H != None:
        df.loc[next_game_id_H, 'H_RTG_home'] = new_ratings['H_RTG_home']
        df.loc[next_game_id_H, 'H_RTG_away'] = new_ratings['H_RTG_away']
    
    if next_game_id_A != None:
        df.loc[next_game_id_A, 'A_RTG_home'] = new_ratings['A_RTG_home']
        df.loc[next_game_id_A, 'A_RTG_away'] = new_ratings['A_RTG_away']
    
    # TODO: Gucken ob die Indizes passen oder ob da kleine errors drin sind die alles durcheinander wirbeln
    

### Expected goal difference for the match

In [70]:
for row, game in results.iterrows():
    exp_gd_match = predict_gd(df.loc[row, 'H_RTG_home']) - predict_gd(df.loc[row, 'A_RTG_away'])
    df.loc[row, 'EGD'] = exp_gd_match

## PageRank

To save computation time, we don't calculate an entire matrix every time. Instead, we just calculate the average points of the two given teams against each other from the last two years (like described in the Hubacek paper). 

In [20]:
# We initialize it with 1 point per game as the default value.
df['EPTS_PR_home'] = 1
df['EPTS_PR_away'] = 1

for row, game in results.iterrows():
    home_id = game['HomeID']
    away_id = game['AwayID']
    
    game_date = game['Date']

    hist_games = get_historical_games(home_id, game_date)  # Could also use away_id here, it doesn't matter

    # Previous matches of the two current teams
    previous_matches = hist_games[((home_id == hist_games['HomeID']) | (home_id == hist_games['AwayID']))
              & ((away_id == hist_games['HomeID']) | (away_id == hist_games['AwayID']))]
    
    game_count = len(previous_matches)
    
    if game_count == 0:
        continue              # Avoid division by zero by just letting the PR for both teams be 1
    
    # Number of wins of the current(!) home team in previous_matches
    win_count_H = len(previous_matches[((previous_matches['HomeID'] == home_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == home_id) & previous_matches['FTR'] == 2)])
    
    # Number of draws in previous_matches
    draw_count = len(previous_matches[previous_matches['FTR'] == 0])
    
    # Number of wins of the current(!) away team in previous_matches
    win_count_A = len(previous_matches[((previous_matches['HomeID'] == away_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == away_id) & previous_matches['FTR'] == 2)])
    
    df.loc[row, 'EPTS_PR_home'] = (3 * win_count_H + draw_count) / game_count
    df.loc[row, 'EPTS_PR_away'] = (3 * win_count_A + draw_count) / game_count

# Implementing own features

**In the following section, some of my own feature ideas are implemented into the dataframe.**

These can be based on historical data, current form data, league data or whatever else.

## Red Cards

Here, we accumulate the red cards each team collected in the last three games. Missing a player due to a red card can be a potentially important factor.

In [71]:
df['RED_home'] = 0
df['RED_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']
        
        recent = get_last_n_games(team_id, game_date, n=3)
        
        suffix = suffixes[id_name]
        
        df.loc[row, 'RED' + suffix] = recent['HR'].sum()

## Difference between own shots and shots conceded (historical)

In [72]:
df['H_ST_home'] = 0
df['A_ST_home'] = 0
df['H_ST_away'] = 0
df['A_ST_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        hist_games = get_historical_games(team_id, game_date)
        
        home_shots_mean = hist_games[hist_games['HomeID'] == team_id]['HS'].mean()
        away_shots_mean = hist_games[hist_games['AwayID'] == team_id]['AS'].mean()
        
        if np.isnan(home_shots_mean):
            home_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        if np.isnan(away_shots_mean):
            away_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        df.loc[row, 'H_ST' + suffixes[id_name]] = home_shots_mean
        df.loc[row, 'A_ST' + suffixes[id_name]] = away_shots_mean

## Match importance (similar to Hubacek)

### Season

In [73]:
# Returns the season corresponding to a certain date.
def get_season_from_date(date):
    for i in range(9):
        if date < datetime.date(2010 + i+1, 7, 15):
            return i+1

In [74]:
# To build up tables, there needs to be a cut where one season ends and the next begins.
# This is the 15th of July every year here.
df['season'] = 0

for row, game in results.iterrows():
    game_date = game['Date']
    #print(get_season_from_date(game_date))
    df.loc[row, 'season'] = get_season_from_date(game_date)

### Round (1-34 / 1-38)

In [177]:
df['RND'] = 0

prev_season = 1
cur_matchday = 1

modulo_add = 1
idx_add = 0

for i in range(len(df.index)):
    game_season = df.loc[i, 'season']

    # This fixes the matchday for games that were moved from their original dates
    if i == 6969:
        idx_add += 1
    if i == 7931:
        idx_add += 1
    if i == 7939:
        idx_add += 1
    if i == 7981:
        df.loc[i, 'RND'] = 24
        idx_add -= 1
        continue
    if i == 7992:
        df.loc[i, 'RND'] = 25
        idx_add -= 1
        continue
    if i == 8467:
        idx_add += 1
    if i == 8474:
        idx_add += 1
    if i == 8611:
        df.loc[i, 'RND'] = 3
        idx_add -= 1
        continue
    
    if game_season != prev_season:       # check if new season has begun
        cur_matchday = 1
        #idx_add = 0

    matches_per_md = int(np.round(df.loc[i, 'TEAM_CNT_league'] / 2))

    df.loc[i, 'RND'] = cur_matchday

    # Change modulo_add for the switch from Germany (9 games per matchday) to Italy (10 games per matchday).
    # Used to avoid getting bugs in current matchday.
    if i == 6174:
        modulo_add = -3
        
    if (i + idx_add + modulo_add) % matches_per_md == 0:      # check if last game of a matchday has been played
        cur_matchday += 1

    prev_season = game_season

### Current standings

In [32]:
get_last_n_games(8, datetime.date(2010,8,22), n=1, df_mode=True)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,RED_away,H_ST_home,H_ST_away,A_ST_home,A_ST_away,season,RND,Date,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
8,8.0,10.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2010-08-15,1,1


In [182]:
# Points after the current game
df['PTS_AFTER_GAME_home'] = 0
df['PTS_AFTER_GAME_away'] = 0

for row, game in df.iterrows():
    #print(row)
    # Game result
    game_res = game['FTR']
    game_date = game['Date']
    
    # Get previous points of both teams (0 if this game is part of the first matchday)
    if game['RND'] == 1:
        prev_points_home = 0
        prev_points_away = 0
    else:
        # Last game played by the home team
        last_game_home = get_last_n_games(game['HomeID'], game_date, n=1, df_mode=True)
        # Last game played by the away team
        last_game_away = get_last_n_games(game['AwayID'], game_date, n=1, df_mode=True)
        
        #print(last_game_home.iloc[0])
        #print(row)
        
        # Get points from after the teams' last games
        if game['HomeID'] == last_game_home.iloc[0]['HomeID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_home']
        elif game['HomeID'] == last_game_home.iloc[0]['AwayID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_away']
        
        if game['AwayID'] == last_game_away.iloc[0]['HomeID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_home']
        elif game['AwayID'] == last_game_away.iloc[0]['AwayID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_away']
        
    if game_res == 1:      # Home win
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 3
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away
    elif game_res == 0:    # Draw
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 1
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 1
    elif game_res == 2:    # Away win
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 3
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home

In [119]:
#df[6950:7000]
#df[6960+330:7000+330]

In [183]:
df[9190-730:9230-730]
#df[8470+350:8490+360]

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,RED_away,H_ST_home,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
8460,67,86,3,2,1,2016-08-21,0.473684,0.289474,0.263158,0.368421,...,1,15.131579,11.157895,14.657895,12.000000,7,1,I1,3,0
8461,68,89,0,1,2,2016-08-21,0.368421,0.263158,0.210526,0.315789,...,0,12.947368,10.500000,13.631579,10.921053,7,1,I1,0,3
8462,85,81,2,2,0,2016-08-21,0.000000,0.000000,0.000000,0.000000,...,1,0.000000,0.000000,18.921053,14.710526,7,1,I1,1,1
8463,79,78,0,1,2,2016-08-27,0.578947,0.105263,0.384615,0.282051,...,0,15.289474,12.410256,17.717949,13.947368,7,2,I1,3,6
8464,81,67,4,2,1,2016-08-27,0.710526,0.210526,0.410256,0.230769,...,2,18.921053,14.897436,15.025641,11.157895,7,2,I1,4,3
8465,72,62,2,2,0,2016-08-28,0.210526,0.210526,0.200000,0.300000,...,0,13.684211,12.000000,15.897436,13.500000,7,2,I1,1,4
8466,93,77,1,3,2,2016-08-28,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,5.000000,13.692308,11.684211,7,2,I1,0,6
8467,66,65,1,0,1,2016-08-28,0.526316,0.289474,0.410256,0.230769,...,0,18.052632,13.025641,12.871795,10.289474,7,2,I1,3,3
8468,74,68,1,1,0,2016-08-28,0.526316,0.236842,0.358974,0.282051,...,2,16.394737,13.435897,12.794872,10.500000,7,2,I1,1,1
8469,70,82,2,1,1,2016-08-28,0.394737,0.368421,0.230769,0.333333,...,0,12.973684,11.564103,13.948718,10.315789,7,2,I1,6,0


In [178]:
#len(df[df['RND'] == 39])
df[df['RND'] == 39]

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,RED_away,H_ST_home,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away


In [21]:
# Calculate table by points from of last matchday

In [112]:
# Gets current position of a team AFTER a given matchday in a given season
def get_cur_standings(team_id, season, matchday, league):
    # Firstly get all the games on that matchday
    games_on_md = df[(df['season'] == season) & (df['RND'] == matchday) & (df['league'] == league)]
    
    cur_standings = pd.DataFrame(columns=['team_id', 'pts'])
    
    for i in [x * 2 for x in list(range(len(games_on_md)))]:
        cur_standings.loc[i, 'team_id'] = games_on_md['HomeID'].iloc[int(i/2)]
        cur_standings.loc[i, 'pts'] = games_on_md['PTS_AFTER_GAME_home'].iloc[int(i/2)]
        cur_standings.loc[i+1, 'team_id'] = games_on_md['AwayID'].iloc[int(i/2)]
        cur_standings.loc[i+1, 'pts'] = games_on_md['PTS_AFTER_GAME_away'].iloc[int(i/2)]
    
    cur_standings.sort_values(by='pts', axis=0, ascending=False, inplace=True)
    
    cur_standings.index = range(1, len(cur_standings.index) + 1)
    
    return cur_standings

In [116]:
get_cur_standings(39, 5, 20, 'D1')

,team_id,pts
1,47,0
2,41,0
3,49,0
4,45,0
5,53,0
6,40,0
7,50,0
8,35,0
9,52,0
10,54,0


In [45]:
# Returns a tuple of a team's position in the standings and the team's points AFTER a given matchday in a given season
def get_cur_pos_and_points(team_id, season, matchday, league):
    game = df[((df['HomeID'] == team_id ) | (df['AwayID'] == team_id))
             & (df['season'] == season) & (df['RND'] == matchday)]
    
    cur_team_position = get_cur_position(team_id, season, matchday, league)
    
    if team_id == game['HomeID']:
        return (game['PTS_AFTER_GAME_home'], cur_team_position)
    elif team_id == game['AwayID']:
        return (game['PTS_AFTER_GAME_away'], cur_team_position)
    else:
        return np.nan

In [ ]:
get_cur_pos_and_points(69, 5, 24, 'I1')

In [158]:
df.sample(10)

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,RED_away,H_ST_home,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
11320,98,109,1,0,1,2015-02-02,0.459459,0.216216,0.263158,0.263158,...,0,13.270270,10.078947,14.526316,12.243243,5,21,SP1,0,0
3665,42,39,4,1,1,2011-04-02,0.615385,0.307692,0.785714,0.071429,...,0,17.846154,16.142857,11.071429,9.153846,1,28,D1,0,0
8644,81,85,3,1,1,2017-01-15,0.769231,0.179487,0.421053,0.210526,...,2,19.256410,14.815789,12.444444,11.500000,7,20,I1,0,0
9649,108,100,6,1,1,2010-10-03,1.000000,0.000000,0.333333,0.666667,...,1,22.500000,21.000000,16.333333,13.000000,1,6,SP1,0,0
6696,67,83,2,0,1,2011-12-17,0.692308,0.230769,0.538462,0.307692,...,1,17.423077,13.384615,11.714286,10.285714,2,15,I1,0,0
5170,52,40,5,1,1,2016-03-05,0.371429,0.171429,0.205882,0.323529,...,0,14.342857,12.147059,15.029412,11.971429,6,25,D1,0,0
676,21,12,0,2,2,2012-03-24,0.428571,0.428571,0.266667,0.200000,...,0,11.071429,9.600000,14.500000,10.727273,2,30,E1,0,0
7629,79,70,2,0,1,2014-04-06,0.605263,0.210526,0.256410,0.256410,...,0,15.473684,11.846154,13.771429,10.794118,4,32,I1,0,0
12766,120,125,2,4,2,2018-12-01,0.333333,0.333333,0.285714,0.428571,...,0,12.500000,9.000000,11.657895,9.947368,9,14,SP1,0,0
3759,46,53,1,0,1,2011-08-27,0.500000,0.111111,0.263158,0.315789,...,0,14.722222,12.000000,11.500000,13.000000,2,4,D1,0,0


In [4]:
df.columns

Index(['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR', 'Date', 'H_WIN_PCT_home',
       'H_DRAW_PCT_home', 'A_WIN_PCT_home', 'A_DRAW_PCT_home',
       'H_WIN_PCT_away', 'H_DRAW_PCT_away', 'A_WIN_PCT_away',
       'A_DRAW_PCT_away', 'H_GS_AVG_home', 'H_GC_AVG_home', 'A_GS_AVG_home',
       'A_GC_AVG_home', 'H_GS_AVG_away', 'H_GC_AVG_away', 'A_GS_AVG_away',
       'A_GC_AVG_away', 'H_GS_STD_home', 'H_GC_STD_home', 'A_GS_STD_home',
       'A_GC_STD_home', 'H_GS_STD_away', 'H_GC_STD_away', 'A_GS_STD_away',
       'A_GC_STD_away', 'WIN_PCT_home', 'WIN_PCT_away', 'DRAW_PCT_home',
       'DRAW_PCT_away', 'GS_AVG_home', 'GC_AVG_home', 'GS_AVG_away',
       'GC_AVG_away', 'GS_STD_home', 'GC_STD_home', 'GS_STD_away',
       'GC_STD_away', 'REST_home', 'REST_away', 'H_GS_AVG_league',
       'A_GS_AVG_league', 'H_GS_STD_league', 'A_GS_STD_league',
       'H_WIN_PCT_league', 'DRAW_PCT_league', 'TEAM_CNT_league',
       'GD_STD_league', 'RND_CNT_league', 'H_RTG_home', 'A_RTG_home',
       'H_RTG_away', '

# Save dataframe as .pkl

In [87]:
df.to_pickle('feature_frame.pkl')